In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean, when

In [2]:
spark = SparkSession.builder.appName("Clients Analysis").getOrCreate() 

In [3]:
# Chargement du  CSV
df = spark.read.csv("clients.csv", header=True, inferSchema=True)

In [4]:
df.show()

+------+----+---------+------+
|   Nom| Age|    Ville|Revenu|
+------+----+---------+------+
| Alice|  25|    Paris| 50000|
|   Bob|NULL|     Lyon| 40000|
|Claire|  35|Marseille| 35000|
| David|  40|     NULL| 45000|
|  Emma|  22|     Lyon|  NULL|
+------+----+---------+------+



In [7]:
# Remplissage des valeurs manquantes pour l’âge avec la moyenne des âges
avg_age = df.select(mean(col("Age"))).first()[0]
print(avg_age)
df = df.withColumn("Age", when(col("Age").isNull(), avg_age).otherwise(col("Age")))

30.5


In [8]:
df.show()

+------+----+---------+------+
|   Nom| Age|    Ville|Revenu|
+------+----+---------+------+
| Alice|25.0|    Paris| 50000|
|   Bob|30.5|     Lyon| 40000|
|Claire|35.0|Marseille| 35000|
| David|40.0|     NULL| 45000|
|  Emma|22.0|     Lyon|  NULL|
+------+----+---------+------+



In [9]:
# Remplissage des valeurs manquantes pour la ville avec "Inconnue"
df = df.fillna({"Ville": "Inconnue"})

In [10]:
df.show()

+------+----+---------+------+
|   Nom| Age|    Ville|Revenu|
+------+----+---------+------+
| Alice|25.0|    Paris| 50000|
|   Bob|30.5|     Lyon| 40000|
|Claire|35.0|Marseille| 35000|
| David|40.0| Inconnue| 45000|
|  Emma|22.0|     Lyon|  NULL|
+------+----+---------+------+



In [11]:
# Suppression des lignes où le revenu est manquant
df = df.filter(col("Revenu").isNotNull())

In [12]:
df.show()

+------+----+---------+------+
|   Nom| Age|    Ville|Revenu|
+------+----+---------+------+
| Alice|25.0|    Paris| 50000|
|   Bob|30.5|     Lyon| 40000|
|Claire|35.0|Marseille| 35000|
| David|40.0| Inconnue| 45000|
+------+----+---------+------+



In [13]:
spark.stop()